In [2]:
import os

if not os.path.isdir('SmartElementData'):
    !git clone https: // github.com / EwaFengler / SmartElementData
else:
    print('skipping')

skipping


In [2]:
from os import listdir
from os.path import isfile, join
import numpy as np

import pandas as pd
import tensorflow as tf

In [6]:
data_path = 'SmartElementData/'

DATA_SERIES = 3

SIZE = DATA_SERIES * 20

In [4]:
def moves_generator(path, y):
    def prepare_moves():
        files = [join(path, f) for f in listdir(path) if isfile(join(path, f))]

        for file in files:
            data = pd.read_csv(file, sep=' ', usecols=[0, 1, 2, 6], names=['time', 'x', 'y', 'zg'])

            data["time"] //= 1000000
            #16 observations per second
            data["time"] //= 62.5
            data = data.groupby('time').mean()

            data = data[:19].to_numpy().flatten()
            yield data, y

    return prepare_moves

In [5]:
def random_generator(path, step):
    def prepare_random():
        files = [join(path, f) for f in listdir(path) if isfile(join(path, f))]

        for file in files:
            data = pd.read_csv(file, sep=' ', usecols=[0, 1, 2, 6], names=['time', 'x', 'y', 'zg'])

            data["time"] //= 1000000
            #16 observations per second
            data["time"] //= 62.5
            data = data.groupby('time').mean()

            i = 0
            while i + 20 < len(data):
                x = data[i:i + 19].to_numpy().flatten()
                y = [0, 0, 0]
                if len(x) == SIZE:
                    yield x, y
                i += step

    return prepare_random


In [8]:
params = {'output_types': (tf.float32, tf.int64), 'output_shapes': ((SIZE,), (3,))}
ds_eh = tf.data.Dataset.from_generator(moves_generator(data_path + 'E/horizontal', [1, 0, 0]), **params)
ds_dh = tf.data.Dataset.from_generator(moves_generator(data_path + 'D/horizontal', [1, 0, 0]), **params)
ds_ev = tf.data.Dataset.from_generator(moves_generator(data_path + 'E/vertical', [0, 1, 0]), **params)
ds_dv = tf.data.Dataset.from_generator(moves_generator(data_path + 'D/vertical', [0, 1, 0]), **params)
ds_ef = tf.data.Dataset.from_generator(moves_generator(data_path + 'E/forward', [0, 0, 1]), **params)
ds_df = tf.data.Dataset.from_generator(moves_generator(data_path + 'D/forward', [0, 0, 1]), **params)
ds_er = tf.data.Dataset.from_generator(random_generator(data_path + 'E/random', 5), **params)
ds_dr = tf.data.Dataset.from_generator(random_generator(data_path + 'D/random', 5), **params)


In [22]:
true_move_ds = [ds_eh, ds_dh, ds_ev, ds_dv, ds_ef, ds_df]

ds_train = ds_er.skip(50).concatenate(ds_dr.skip(50))
for ds in true_move_ds:
    ds_train = ds_train.concatenate(ds.skip(15))
ds_train = ds_train.shuffle(2000).batch(32)

ds_test = ds_er.take(50).concatenate(ds_dr.take(50))
for ds in true_move_ds:
    ds_test = ds_test.concatenate(ds.take(15))
ds_test = ds_test.shuffle(2000).batch(32)

ds_all = ds_er.concatenate(ds_dr)
for ds in true_move_ds:
    ds_all = ds_all.concatenate(ds)
ds_all = ds_all.shuffle(2000).batch(32)

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, 'sigmoid')
], name='model')

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['binary_accuracy'])

model.fit(ds_train, epochs=10, validation_data=ds_test, shuffle=True)

Epoch 1/10
27/27 [==============================] - 4s 30ms/step - loss: 0.9518 - binary_accuracy: 0.7055 - val_loss: 0.3421 - val_binary_accuracy: 0.8544
Epoch 2/10
27/27 [==============================] - 4s 26ms/step - loss: 0.4175 - binary_accuracy: 0.8283 - val_loss: 0.2967 - val_binary_accuracy: 0.9035
Epoch 3/10
27/27 [==============================] - 4s 26ms/step - loss: 0.2787 - binary_accuracy: 0.8815 - val_loss: 0.2347 - val_binary_accuracy: 0.9351
Epoch 4/10
27/27 [==============================] - 4s 26ms/step - loss: 0.2265 - binary_accuracy: 0.9106 - val_loss: 0.1998 - val_binary_accuracy: 0.9404
Epoch 5/10
27/27 [==============================] - 4s 25ms/step - loss: 0.1811 - binary_accuracy: 0.9308 - val_loss: 0.1851 - val_binary_accuracy: 0.9439
Epoch 6/10
27/27 [==============================] - 4s 27ms/step - lo

In [24]:
model.predict(ds_df.take(1).batch(1))

array([[2.6521087e-04, 2.3554308e-06, 9.7975791e-01]], dtype=float32)

In [25]:
model.fit(ds_all, epochs=10, shuffle=True)

Epoch 1/10
33/33 [==============================] - 3s 1ms/step - loss: 0.0883 - binary_accuracy: 0.9695
Epoch 2/10
33/33 [==============================] - 3s 1ms/step - loss: 0.0609 - binary_accuracy: 0.9819
Epoch 3/10
33/33 [==============================] - 3s 1ms/step - loss: 0.0532 - binary_accuracy: 0.9841
Epoch 4/10
33/33 [==============================] - 3s 1ms/step - loss: 0.0520 - binary_accuracy: 0.9831
Epoch 5/10
33/33 [==============================] - 3s 1ms/step - loss: 0.0434 - binary_accuracy: 0.9847
Epoch 6/10
33/33 [==============================] - 3s 1ms/step - loss: 0.

In [26]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               6100      
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                6464      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99    

In [ ]:
model.save('MLP_model', include_optimizer=False)

INFO:tensorflow:Assets written to: MLP_model/assets


INFO:tensorflow:Assets written to: MLP_model/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

f = open('model_xy_zgravity.tflite', 'wb')
f.write(tflite_model)
f.close()

STASH:


In [ ]:
assert False, "do not run rest of cells when using 'run all'"

AssertionError: ignored

In [ ]:
from ipywidgets import interact, IntSlider

H = [join("horizontal", f) for f in listdir("horizontal") if isfile(join("horizontal", f))]


@interact(i=IntSlider(min=0, max=len(H) - 1))
def show(i):
    data = pandas.read_csv(H[i], sep=" ", names=["time", "x", "y", "z", "a", "p", "r"])

    data["time"] //= 1000000

    #16 obserwacji na 1s
    data["time"] //= 62.5
    data = data.groupby('time').mean()

    print(data[:19].to_numpy().flatten().dtype)

#     plt.figure()
#     data['x'].plot()
#     data['y'].plot()
#     data['z'].plot()


In [ ]:
def approxByKPoints(x, k):
    points = [int(round(p)) for p in np.linspace(0, len(x), k + 1)]

    res = [np.mean(x[points[i]:points[i + 1]]) for i in range(k)]

    return res


print(approxByKPoints([1, 2, 3, 4, 5, 4, 3, 2, 1, 2], 3))